# DATA

In [ ]:
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))



Found GPU at: /device:GPU:0


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd
import sklearn
import os
import pickle
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV, ParameterGrid
from sklearn.preprocessing import StandardScaler
from datetime import datetime

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1'  
tf.random.set_seed(1234)

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/THESIS/data/final_data.csv')
data.head()

,r,sigma,T,s0,k,t,asset,call,asset_greater_call,scaled_call,scaled_asset,tau,moneyness,d1,d2,delta,gamma,theta,vega,rho
0,0.05,0.05,0.1,10.0,10.0,0.0,10.000000,0.090978,True,0.009098,1.0,0.10,1.000000,0.324133,0.308322,0.627082,2.394010,-0.608243,1.197005,0.617984
1,0.05,0.05,0.1,10.0,10.0,1.0,9.977024,0.071240,True,0.007124,1.0,0.09,0.997702,0.154153,0.139153,0.561255,2.634254,-0.604191,1.179973,0.497558
2,0.05,0.05,0.1,10.0,10.0,2.0,9.979236,0.066344,True,0.006634,1.0,0.08,0.997924,0.142939,0.128797,0.556831,2.798086,-0.622830,1.114591,0.439232
3,0.05,0.05,0.1,10.0,10.0,3.0,10.085137,0.133263,True,0.013326,1.0,0.07,1.008514,0.912040,0.898811,0.819126,1.972797,-0.657203,0.702286,0.568942
4,0.05,0.05,0.1,10.0,10.0,4.0,9.964273,0.045868,True,0.004587,1.0,0.06,0.996427,-0.041160,-0.053407,0.483584,3.266261,-0.644005,0.972889,0.286362


In [ ]:
scaler = StandardScaler()
X = data[['r','sigma','tau','k','asset']]
y = data['call']
#X = scaler.fit_transform(X)
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=1234)

# NEURAL NET MODEL

In [ ]:
pip install -q -U tensorflow_addons

     |████████████████████████████████| 1.1 MB 16.3 MB/s 


In [ ]:
from keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam
from keras.layers import Embedding, LSTM, Dense, BatchNormalization, Input, Flatten, Dropout
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.wrappers.scikit_learn import KerasRegressor
from tensorflow.keras.initializers import RandomUniform, GlorotUniform, HeUniform
from keras import backend as K
import tensorflow_addons as tfa

In [ ]:
def create_model(activation, lr_0, batch_norm, dropout_rate, layer_number, neuron_number,\
                neuron_decrease, data_length, initializer):
    opt = Adam(learning_rate = lr_0)
    model = Sequential()
    model.add(Dense(neuron_number, input_shape=(5,), activation = activation, \
                    kernel_initializer=initializer , bias_initializer=initializer))
    for i in range(layer_number):
        if batch_norm == True:
            model.add(BatchNormalization())         
        neuron_number = int(neuron_number/neuron_decrease)
        model.add(Dense(neuron_number, activation = activation))
    if batch_norm == True:
        model.add(BatchNormalization()) 
    model.add(Dropout(dropout_rate))
    model.add(Dense(1, name='Final_1D_output', activation = activation))
    model.compile(optimizer=opt,loss='mean_squared_error',\
                  metrics=[tfa.metrics.RSquare(dtype=tf.float32, y_shape=(1,))],\
                 run_eagerly=True)
    print(model.summary())
    return model

def callback_list(patience):
    early_stop = EarlyStopping(monitor='val_loss', patience=patience, restore_best_weights=True)
    return early_stop   

In [ ]:
callbacks = callback_list(patience=2)

# HYPERPARAMETER TUNING: SKLEARN

In [ ]:
sklearn_model = KerasRegressor(build_fn = create_model)

params = dict(activation=['relu'],lr_0 = [0.001,0.0001,0.00001], batch_norm=[True, False], \
              dropout_rate = [0.0,0.1], layer_number = [3], neuron_decrease = [1,2], \
              neuron_number = [512,256,128,64], data_length = [X.shape[0]], batch_size = [512,1024,2048,4096], \
              initializer = [RandomUniform(), GlorotUniform()] )

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  """Entry point for launching an IPython kernel.


In [ ]:
len(ParameterGrid(params))

768

In [ ]:
random_search = RandomizedSearchCV(sklearn_model, param_distributions=params, n_iter = 100, cv=3, verbose=3) 

We include fitting and saving of the model in the same cell, so we can leave it running in background with the Colab Pro + feature. We save model, its weights and the optimal parameters, so we can use them in another notebook, with the already trained NN. Finally, we print the total running time in seconds.

In [ ]:
start = datetime.now()

random_search_results = random_search.fit(X_train,y_train,callbacks=[callbacks],validation_split=0.2,\
                                         epochs = 50)

end = datetime.now()

# serialize model to JSON
model_json = random_search_results.best_estimator_.model.to_json()
with open("/content/drive/MyDrive/THESIS/results_search/model_greeks.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
random_search_results.best_estimator_.model.save_weights("/content/drive/MyDrive/THESIS/results_search/model_greeks.h5")
print("Saved model to disk")

import pickle
search_params = random_search.best_params_

a_file = open("/content/drive/MyDrive/THESIS/results_search/search_params_greeks.pkl", "wb")
pickle.dump(search_params, a_file)
a_file.close()
print("Saved params to disk")

model_hist = random_search_results.best_estimator_.model.history
train_hist = pd.DataFrame(model_hist.history)
train_hist.to_csv('/content/drive/MyDrive/THESIS/results_search/train_hist_greeks.csv', index=False)
print("Saved history to disk")

total_time = (end-start).total_seconds()
print(total_time)

Streaming output truncated to the last 5000 lines.
Epoch 2/50
834/834 [==============================] - 15s 18ms/step - loss: 0.0684 - r_square: 0.9986 - val_loss: 0.0537 - val_r_square: 0.9989
Epoch 3/50
834/834 [==============================] - 15s 18ms/step - loss: 0.0329 - r_square: 0.9993 - val_loss: 0.0191 - val_r_square: 0.9996
Epoch 4/50
834/834 [==============================] - 15s 18ms/step - loss: 0.0247 - r_square: 0.9995 - val_loss: 0.0103 - val_r_square: 0.9998
Epoch 5/50
834/834 [==============================] - 15s 18ms/step - loss: 0.0203 - r_square: 0.9996 - val_loss: 0.0121 - val_r_square: 0.9998
Epoch 6/50
521/521 [==============================] - 5s 9ms/step - loss: 0.0105 - r_square: 0.9998
[CV 1/3] END activation=relu, batch_norm=False, batch_size=512, data_length=1000000, dropout_rate=0.0, initializer=<keras.initializers.initializers_v2.RandomUniform object at 0x7f47198d2250>, layer_number=3, lr_0=0.001, neuron_decrease=1, neuron_number=128;, score=-0.010 t